In [1]:
import numpy as np

# data 

In [2]:
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Model 

In [5]:
import keras

In [6]:
from keras.layers import (Conv2D, Input, Add, Dense, Activation, ZeroPadding2D, 
                          BatchNormalization, Flatten, AveragePooling2D, 
                          MaxPooling2D, GlobalMaxPooling2D)

In [7]:
from keras.models import Model

In [8]:
def identity_block(X, f, filters):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters=F1, kernel_size=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=(f, f), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F3, kernel_size=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [9]:
def conv_block(X, f, filters, s=2):
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    
    X_shortcut = Conv2D(F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [10]:
def cnn_ResNet():
    X_input = Input((32, 32, 3))
    
    X = Conv2D(64, (3, 3), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X) # (32, 32, 64)
    X = MaxPooling2D()(X) # (16, 16, 64)
    print(X.shape)
    
    F = [16, 16, 64]
    X = conv_block(X, f=3, filters=F)
    X = identity_block(X, f=3, filters=F) # (8, 8, 64)    
    print(X.shape)
    
    F = [32, 32, 128]
    X = conv_block(X, f=3, filters=F)
    X = identity_block(X, f=3, filters=F) # (4, 4, 128)    
    print(X.shape)
    
    F = [64, 64, 256]
    X = conv_block(X, f=3, filters=F)
    X = identity_block(X, f=3, filters=F) # (2, 2, 256)    
    print(X.shape)
    
    X = AveragePooling2D(pool_size=(2, 2))(X) # (1, 1, 512)
    
    X = Flatten()(X)
    X = Dense(10, activation='softmax')(X)
    
    model = Model(X_input, X)
    
    return model

In [11]:
model = cnn_ResNet()

(?, 16, 16, 64)
(?, 8, 8, 64)
(?, 4, 4, 128)
(?, 2, 2, 256)


In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 16, 16, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# training

In [13]:
from keras.optimizers import Adam

In [14]:
model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, y_train, batch_size=256, epochs=40, validation_split=0.1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/40
45000/45000 [==============================] - 20s 444us/step - loss: 1.7341 - acc: 0.3714 - val_loss: 1.7314 - val_acc: 0.3886
Epoch 2/40
45000/45000 [==============================] - 6s 143us/step - loss: 1.3805 - acc: 0.4996 - val_loss: 1.3872 - val_acc: 0.5008
Epoch 3/40
45000/45000 [==============================] - 6s 142us/step - loss: 1.2244 - acc: 0.5579 - val_loss: 1.3304 - val_acc: 0.5138
Epoch 4/40
45000/45000 [==============================] - 6s 141us/step - loss: 1.1184 - acc: 0.5990 - val_loss: 1.3104 - val_acc: 0.5428
Epoch 5/40
45000/45000 [==============================] - 6s 141us/step - loss: 1.0262 - acc: 0.6312 - val_loss: 1.2915 - val_acc: 0.5548
Epoch 6/40
45000/45000 [==============================] - 6s 142us/step - loss: 0.9473 - acc: 0.6620 - val_loss: 1.2521 - val_acc: 0.5696
Epoch 7/40
45000/45000 [==============================] - 6s 141us/step - loss: 0.8771 - acc: 0.6860 - val_loss: 1.3537 - 

In [16]:
model.evaluate(X_test, y_test, batch_size=256)

10000/10000 [==============================] - 1s 70us/step


[2.4030862201690675, 0.58]